In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Any
import pandas as pd
import ast, os, glob
import kagglehub
from sentence_transformers import SentenceTransformer, util
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from itertools import cycle, islice
import random
import traceback

In [ ]:
nest_asyncio.apply()
ngrok.set_auth_token("2wcLhKAkhn0MwFEwOlt2CXaabqk_4n42xCLtLqGuFKiKLfrik")
public_url = ngrok.connect(8000)
print(f"Swagger docs available at: {public_url}/docs")

Swagger docs available at: NgrokTunnel: "https://f7fd-34-16-154-44.ngrok-free.app" -> "http://localhost:8000"/docs


In [ ]:
# ------------------ 데이터 로딩 및 전처리 ------------------
DATASET_SLUG = "omarxadel/fitness-exercises-dataset"
CACHE_BASE = os.path.expanduser("~/.cache/kagglehub/datasets")
CACHE_PATH = os.path.join(CACHE_BASE, *DATASET_SLUG.split('/'))
if not os.path.exists(CACHE_PATH) or not glob.glob(f"{CACHE_PATH}/*.csv"):
    DATA_PATH = kagglehub.dataset_download(DATASET_SLUG)
else:
    DATA_PATH = CACHE_PATH

csv_files = glob.glob(os.path.join(DATA_PATH, "*.csv"))
if not csv_files:
    raise FileNotFoundError(f"CSV 파일이 {DATA_PATH}에 없습니다.")
CSV_PATH = csv_files[0]
print(f"Loading CSV: {CSV_PATH}")

df = pd.read_csv(CSV_PATH)

Loading CSV: /kaggle/input/fitness-exercises-dataset/exercises.csv


In [ ]:
# ------------------ 전처리 ------------------
# target_parts: target + secondaryMuscles/*
sec_cols = [c for c in df.columns if c.startswith('secondaryMuscles')]

def to_list(x):
    if pd.isna(x): return []
    return [x] if isinstance(x, str) else ([] if not isinstance(x, list) else x)

df['target_parts'] = df['target'].apply(to_list)
for c in sec_cols:
    df['target_parts'] += df[c].apply(to_list)
# equipment
df['equipment'] = df['equipment'].apply(lambda x: [x] if isinstance(x, str) else [])
# youtube
df['youtube'] = df['gifUrl']
# level
lvl_col = next((c for c in df.columns if 'difficulty' in c or c=='level'), None)
df['level'] = df[lvl_col].astype(str).str.lower() if lvl_col else 'beginner'
# type
df['type'] = df['type'] if 'type' in df.columns else 'strength'
# instruction
instr_cols = [c for c in df.columns if c.startswith('instructions/')]
for c in instr_cols: df[c]=df[c].fillna('')
df['instruction'] = df[instr_cols].agg(' '.join, axis=1)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-97' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [ ]:
# ------------------ SBERT 임베딩 ------------------
model = SentenceTransformer('all-MiniLM-L6-v2')
embs = model.encode(df['instruction'].tolist(), convert_to_tensor=True)

goal_to_types = {
    'muscle_gain': ['strength'],
    'fat_loss': ['cardio','HIIT'],
    'maintenance': ['strength','cardio','HIIT']
}
rep_ranges = {'muscle_gain':'4x8','fat_loss':'3x15','maintenance':'3x12'}
# 한글→영어 부위 매핑
kor2eng = {
    '하체':['quadriceps','hamstrings','glutes','calves','adductors','abductors'],
    '전신':None,  # None → full body 체크
    '가슴':['pectoral'],
    '등':['lats','traps','middle back','lower back'],
    '어깨':['deltoids','traps'],
    '팔':['biceps','triceps','forearms'],
    '복부':['abs']
}

In [ ]:
# ------------------ FastAPI 앱 ------------------
app = FastAPI(title="SBERT Workout Recommender")
class Req(BaseModel):
    goal: str
    preferred_parts: List[str]
    level: str
    frequency_per_week: int

def build_query(goal, parts, level, freq):
    return f"{level} user, {freq} times/week focusing on {', '.join(parts)} for {goal}"

@app.post("/recommend", response_model=Dict[str,Any])
def recommend(r: Req):
    try:
        if r.frequency_per_week<1 or not r.preferred_parts: raise HTTPException(400,'빈도와 부위 확인')
        if r.goal not in goal_to_types: raise HTTPException(400,'목표 오류')
        # map parts
        mapped=[]
        for p in r.preferred_parts:
            syn=kor2eng.get(p)
            mapped.append(syn)  # None for full body
        days=r.frequency_per_week
        bins=list(islice(cycle(mapped), days))
        # user emb
        q=build_query(r.goal,r.preferred_parts,r.level,days)
        uemb=model.encode(q,convert_to_tensor=True)
        sims=util.cos_sim(uemb,embs)[0].cpu().numpy()
        df['score']=sims
        used=set()
        schedule={}
        for i,syns in enumerate(bins,1):
            # part mask
            if syns is None:
                mask=df['target_parts'].apply(lambda L: len(L)>=3)
            else:
                mask=df['target_parts'].apply(lambda L:any(x in L for x in syns))
            # type filter
            types=goal_to_types[r.goal]
            # Tier1: level+type
            cand=df[mask & df['type'].isin(types) & (df['level']==r.level.lower())]
            # Tier2: type only
            if cand.empty: cand=df[mask & df['type'].isin(types)]
            # Tier3: just mask
            if cand.empty: cand=df[mask]
            # remove used
            cand=cand[~cand['name'].isin(used)]
            # sample up to 5
            tops=cand.sort_values('score', ascending=False).head(10)
            picks=tops.sample(min(5,len(tops)),random_state=i).to_dict('records')
            for ex in picks: used.add(ex['name'])
            # prepare output
            exs=[]
            for ex in picks:
                exs.append({k:ex[k] for k in ['name','target_parts','equipment','level','type']}
                           |{'sets':rep_ranges[r.goal],'youtube':ex['youtube'],'score':ex['score']})
            schedule[f'Day{i}']={'target_parts':[r.preferred_parts[(i-1)%len(r.preferred_parts)]],'exercises':exs}
        return {'schedule':schedule}
    except Exception:
        traceback.print_exc()
        raise HTTPException(500,'서버 오류')

In [ ]:
if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [197]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [197]
